# Trigger

## CREATE TRIGGER

```sql
CREATE TRIGGER trigger_name {BEFORE|AFTER} {INSERT|UPDATE|DELETE}
ON tbl_name FOR EACH ROW
# trigger statements including NEW and OLD 
# (enclosed by BEGIN and END if there exists multiple statements).
```

* The NEW table is used in INSERT and UPDATE triggers.

* The OLD table is used in DELETE and UPDATE triggers.



## Show triggers

```sql
SHOW TRIGGERS;

SHOW TRIGGERS FROM db_name WHERE `table` = 'tbl_name';

SELECT TRIGGER_NAME, EVENT_OBJECT_TABLE, EVENT_MANIPULATION
FROM INFORMATION_SCHEMA.TRIGGERS
WHERE TRIGGER_SCHEMA = 'db_name' 
ORDER BY ACTION_TIMING; 
```

## Multiple triggers

Use FOLLOWS or PRECEDES.

```sql
CREATE TRIGGER new_trigger
{BEFORE|AFTER} {INSERT|UPDATE|DELETE}
ON tbl_name FOR EACH ROW
{FOLLOWS|PRECEDES} old_trigger
# trigger body
```

## Calling a procedure in a trigger

A trigger cannot call a stored procedure if the procedure has OUT or INOUT parameters or uses dynamic SQL.


This is an emulator for CHECK (https://www.mysqltutorial.org):

```sql
DELIMITER //
CREATE PROCEDURE check_parts(IN cost DECIMAL(10,2), IN price DECIMAL(10,2))
BEGIN
  IF cost < 0 THEN
      SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'check constraint on parts.cost failed';
  END IF;
  
  IF price < 0 THEN
      SIGNAL SQLSTATE '45001' SET MESSAGE_TEXT = 'check constraint on parts.price failed';
  END IF;
  
  IF price < cost THEN
      SIGNAL SQLSTATE '45002' SET MESSAGE_TEXT = 'check constraint on parts.price and parts.cost failed';
  END IF;
END//
DELIMITER ;

DELIMITER //
CREATE TRIGGER parts_before_insert BEFORE INSERT ON parts
FOR EACH ROW
BEGIN
    CALL check_parts(new.cost, new.price);
END//
DELIMITER ;

DELIMITER //
CREATE TRIGGER parts_before_update BEFORE UPDATE ON parts
FOR EACH ROW
BEGIN
    CALL check_parts(new.cost, new.price);
END//
DELIMITER ;
```

# VIEW

## CREATE or ALFTER VIEW

```sql
CREATE VIEW view_name AS 
SELECT ...
FROM tbl;

CREATE VIEW view_name (x, y) AS
SELECT x1, y1
FROM tbl;
```

When changing a current view,

```sql
ALTER VIEW my_view AS SELECT ...;
```


`ORDER BY` is not allowed in creating a view, unless TOP, OFFSET-FETCH, and FOR XML is used.

## View options

### WITH CHECK OPTION

Some views are updatable. That is, you can use them in statements such as UPDATE, DELETE, or INSERT to update the contents of the underlying table. For a view to be updatable, there must be a one-to-one relationship between the rows in the view and the rows in the underlying table. There are also certain other constructs that make a view nonupdatable.

The `WITH CHECK OPTION` clause can be given for an updatable view to prevent inserts or updates to rows except those for which the WHERE clause in the select_statement is true.

```sql
CREATE TABLE tbl (
    x INT,
    y INT
);

CREATE VIEW tbl_view1 AS
    SELECT x, y
    FROM tbl
    WHERE x>0 AND x<y;
    
INSERT INTO tbl_view1 (x,y) VALUES(10,20);       # (10,20) is inserted into both tbl and tbl_view1.
INSERT INTO tbl_view1 (x,y) VALUES(20,10);       # (20,10) is inserted only into tbl.
INSERT INTO tbl_view1 (x,y) VALUES(5,8),(20,4)   # both are inserted into tbl, but only (5,8) is inserted to tbl_view1.


CREATE VIEW tbl_view2 AS                # Not updatable.             
    SELECT x, x + y
    FROM tbl;


CREATE VIEW tbl_view3 AS
    SELECT x, y
    FROM tbl
    WHERE x>0 AND x<y
WITH CHECK OPTION;

INSERT INTO tbl_view3 (x,y) VALUES(10,20);       # (10,20) is inserted into both tbl and tbl_view1.
INSERT INTO tbl_view3 (x,y) VALUES(20,10);       # (20,10) is not inserted to tbl either tbl_view3.
```


